In [ ]:
import typing as t
from shapely.geometry import Polygon
from structuralcodes.core.base import ConstitutiveLaw, Material
from structuralcodes.geometry import Geometry


class ShellGeometry(Geometry):
    """Class for representing the geometry of a shell section with reinforcement visualization."""

    def __init__(
        self,
        thickness: float,
        material: t.Union["Material", "ConstitutiveLaw"],
        reinforcement: t.Optional[t.List["ShellReinforcement"]] = None,
        density: t.Optional[float] = None,
        concrete: bool = False,
        name: t.Optional[str] = None,
        group_label: t.Optional[str] = None,
    ) -> None:
        """Initialize a ShellGeometry object."""
        super().__init__(name=name, group_label=group_label)

        if thickness <= 0:
            raise ValueError("Shell thickness must be a positive number.")

        if not isinstance(material, (Material, ConstitutiveLaw)):
            raise TypeError("Material must be a valid Material or ConstitutiveLaw object.")

        self._thickness = thickness
        self._material = material
        self._density = density if density is not None else (
            material.density if isinstance(material, Material) else None
        )
        self._concrete = concrete
        self._reinforcement = reinforcement if reinforcement else []

    @property
    def thickness(self) -> float:
        """Returns the thickness of the shell."""
        return self._thickness

    @property
    def material(self) -> "ConstitutiveLaw":
        """Returns the material of the shell."""
        return self._material

    @property
    def density(self) -> float:
        """Returns the density of the shell material."""
        return self._density

    @property
    def reinforcement(self) -> t.List["ShellReinforcement"]:
        """Returns the reinforcement layers within the shell."""
        return self._reinforcement

    @property
    def concrete(self) -> bool:
        """Returns True if the shell is made of concrete."""
        return self._concrete

    def add_reinforcement(self, reinforcement: "ShellReinforcement") -> None:
        """Adds a reinforcement layer to the shell."""
        self._reinforcement.append(reinforcement)

    def _repr_svg_(self) -> str:
        """Returns an SVG representation of the shell geometry, including reinforcement."""
        width = 1000  # mm (full width of shell)
        height = self.thickness  # mm (shell thickness)

        # SVG elements
        svg_elements = []

        # SVG Header
        svg_elements.append(
            f'<svg width="{width+100}" height="{height+100}" '
            f'viewBox="{-width/2-50} {-height/2-50} {width+100} {height+100}" '
            f'xmlns="http://www.w3.org/2000/svg">'
        )

        # Draw Concrete Section (Gray Rectangle)
        svg_elements.append(
            f'<rect x="{-width/2}" y="{-height/2}" '
            f'width="{width}" height="{height}" '
            f'fill="lightgray" stroke="black" stroke-width="2"/>'
        )

        # Draw Reinforcement Bars (Spread across 1000mm width)
        for layer in self._reinforcement:
            position_y = -height / 2 + layer.position_over_thickness * height  # Convert normalized position
            bar_spacing = layer.center_distance
            bar_diameter = layer.bar_diameter
            num_bars = int(width // bar_spacing)  # Compute number of bars across 1000mm width

            # Compute rebar positions along width
            rebar_positions = [
                (-width / 2 + bar_spacing / 2 + i * bar_spacing, position_y)
                for i in range(num_bars)
            ]

            # Draw rebars as circles
            for x, y in rebar_positions:
                svg_elements.append(
                    f'<circle cx="{x}" cy="{y}" r="{bar_diameter/2}" '
                    f'fill="red" stroke="black" stroke-width="1"/>'
                )


        # Close SVG
        svg_elements.append("</svg>")

        return "".join(svg_elements)


: 

In [ ]:
import typing as t

from structuralcodes.core.base import Section

"""Shell section implemenetation."""
class ShellSection(Section):
    """This is the implementation of the shell section class.

    The section represents a shell element characterized by its thickness,
    material properties, and reinforcement layout.

    Attributes:
        geometry (ShellGeometry): The geometry of the shell, defined by its
            thickness and material.
        name (str): The name of the section.
        section_calculator (ShellSectionCalculator): The object responsible
            for performing calculations on the section (e.g., strain
            integration, moment-curvature analysis).
    """


    def __init__(
        self,
        geometry: 'ShellGeometry',
        name: t.Optional[str] = None,
        integrator: t.Literal['fiber'] = 'fiber',
        **kwargs,
    ) -> None:
        """Initialize a ShellSection.

        Arguments:
            geometry (ShellGeometry): The geometry of the shell section.
            name (str, optional): The name of the section.
            integrator (str): The name of the SectionIntegrator to use.
            kwargs (dict): Additional keyword arguments for the section
                calculator.

        Note:
            The ShellSection uses a ShellSectionCalculator for all
            calculations. This calculator relies on a SectionIntegrator
            for numerical integration. Any additional keyword arguments
            used when creating the ShellSection are passed to the
            ShellSectionCalculator to modify its behaviour.
        """
        if name is None:
            name = 'ShellSection'
        super().__init__(name)

        self.geometry = geometry
        self.section_calculator = ShellSectionCalculator(
            sec = self, integrator = integrator, **kwargs
        )
        self._gross_properties = None

    @property
    def gross_properties(self):
        """Return the gross properties of the shell section."""
        if self._gross_properties is None:
            self._gross_properties = (
                self.section_calculator.calculate_gross_section_properties()
            )
        return self._gross_properties


In [ ]:
from structuralcodes.core.base import Material
import typing as t


class ShellReinforcement:
    """Represents reinforcement in a shell section.

    Attributes:
        position_over_thickness (float): Normalized position in thickness (0 = bottom, 1 = top).
        num_bars (Optional[int]): Number of bars in each bundle (computed if not given).
        center_distance (Optional[float]): Spacing between bar bundles (computed if not given).
        bar_diameter (float): Diameter of individual bars (mm).
        material (Material): The material of the reinforcement.
        orientation (float): Orientation angle in degrees (0° = horizontal, 90° = vertical).
    """

    def __init__(
        self,
        position_over_thickness: float,
        material: "Material",
        bar_diameter: float,
        width: float = 1000,  # Default width for spacing calculation
        num_bars: t.Optional[int] = None,
        center_distance: t.Optional[float] = None,
        orientation: float = 0,  # Default is horizontal
    ) -> None:
        """Initialize a ShellReinforcement object.

        Arguments:
            position_over_thickness (float): Depth position as a fraction of shell thickness.
            material (Material): Reinforcement material.
            bar_diameter (float): Bar diameter (mm).
            width (float): Width of the section (default: 1000mm).
            num_bars (Optional[int]): Number of bars per bundle (if not given, center_distance is used).
            center_distance (Optional[float]): Spacing between bundles (if not given, num_bars is used).
            orientation (float): Angle of reinforcement (degrees).
        """
        if not (0.0 <= position_over_thickness <= 1.0):
            raise ValueError("position_over_thickness must be between 0 and 1.")
        if bar_diameter <= 0:
            raise ValueError("bar_diameter must be positive.")

        self.position_over_thickness = position_over_thickness
        self.bar_diameter = bar_diameter
        self.material = material
        self.orientation = orientation
        self.width = width  # Total width available for spacing

        # Handle center_distance and num_bars
        if num_bars is not None:
            # Compute center_distance if only num_bars is given
            self.num_bars = num_bars
            self.center_distance = width / (num_bars - 1) if num_bars > 1 else width
        elif center_distance is not None:
            # Compute num_bars if only center_distance is given
            self.center_distance = center_distance
            self.num_bars = int(width // center_distance) + 1
        else:
            raise ValueError("Either num_bars or center_distance must be specified.")

        # Ensure both values are now defined before checking positivity
        if self.num_bars <= 0:
            raise ValueError("Computed num_bars must be positive.")
        if self.center_distance is not None and self.center_distance <= 0:
            raise ValueError("Computed center_distance must be positive.")


In [ ]:
from structuralcodes.materials.reinforcement import create_reinforcement
from structuralcodes.materials.concrete import create_concrete
from structuralcodes import set_design_code

set_design_code("ec2_2004")

# Define reinforcement material
reinforcement_material = create_reinforcement(fyk=500, Es=200000, ftk=550, epsuk=0.07)

# Using num_bars (center_distance auto-calculated)
top_layer = ShellReinforcement(
    position_over_thickness=0.1,
    material=reinforcement_material,
    bar_diameter=12,
    num_bars=6  # Will auto-compute center_distance
)

# Using center_distance (num_bars auto-calculated)
bottom_layer = ShellReinforcement(
    position_over_thickness=0.9,
    material=reinforcement_material,
    bar_diameter=16,
    center_distance=120  # Will auto-compute num_bars
)

# Check computed values
print(f"Top Layer -> Bars: {top_layer.num_bars}, Spacing: {top_layer.center_distance} mm")
print(f"Bottom Layer -> Bars: {bottom_layer.num_bars}, Spacing: {bottom_layer.center_distance} mm")

# Create the ShellGeometry instance
geometry = ShellGeometry(
    thickness=500,
    material=create_concrete(fck=30),  
    reinforcement=[top_layer, bottom_layer],
    name="Reinforced Rectangular Section"
)

geometry
